In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline

# Load the data on income and wealth
income_and_wealth = pd.read_excel('data/zh-income-wealth.xlsx')
income_and_wealth.rename(columns={'BFS_NR': 'municipality', 'GEBIET_NAME': 'municipality_name', 'INDIKATOR_JAHR': 'year'}, inplace=True)

# Pick out the series we are interested in : median income
income_and_wealth_grouped = income_and_wealth.groupby('INDIKATOR_ID')
median_income = income_and_wealth_grouped.get_group(316).rename(columns={'INDIKATOR_VALUE' : 'income'})

# Discard the columns we do not care about
median_income = median_income[['municipality', 'year', 'municipality_name', 'income']]

# Load the data on voting
votes = pd.read_csv('data/2015-06-elections-partystrengths/analysis/output/partystrengths.csv')
parties = pd.read_csv('data/2015-06-elections-partystrengths/analysis/output/parties.csv')
parties.rename(columns={'ID': 'party', 'Abbr_D': 'party_name'}, inplace=True)
parties = parties[['party', 'party_name']]

df = median_income.merge(votes,
    on=['municipality', 'year'], how='inner').merge(parties,
    on='party', how='left')